In [1]:
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse
from google.colab import auth
auth.authenticate_user()
from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()
import getpass
!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}

E: Package 'python-software-properties' has no installation candidate
Selecting previously unselected package google-drive-ocamlfuse.
(Reading database ... 110856 files and directories currently installed.)
Preparing to unpack .../google-drive-ocamlfuse_0.7.1-0ubuntu3~ubuntu18.04.1_amd64.deb ...
Unpacking google-drive-ocamlfuse (0.7.1-0ubuntu3~ubuntu18.04.1) ...
Setting up google-drive-ocamlfuse (0.7.1-0ubuntu3~ubuntu18.04.1) ...
Processing triggers for man-db (2.8.3-2ubuntu0.1) ...
Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&response_type=code&access_type=offline&approval_prompt=force
··········
Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=ht

In [0]:
!mkdir -p drive
!google-drive-ocamlfuse drive

In [3]:
!ls

adc.json  drive  sample_data


In [4]:
cd drive/Houston/

/content/drive/Houston


In [0]:
import numpy as np
import cv2

In [0]:
hsi = []
for i in range(3):
  img_path = "pca/hsi_pc" + str(i) + ".png"
  img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
  hsi.append(img)
        
hsi = np.array(hsi)

In [0]:
train_file_name = "labels/train.txt"
test_file_name = "labels/test.txt"

In [0]:
file = open(train_file_name)
triplets = file.read().split()
for i in range(0, len(triplets)):
    triplets[i] = triplets[i].split(",")
train_array = np.array(triplets, dtype=int)
file.close()


file = open(test_file_name)
triplets = file.read().split()
for i in range(0, len(triplets)):
    triplets[i] = triplets[i].split(",")
test_array = np.array(triplets, dtype=int)
file.close()

HEIGHT = train_array.shape[0]
WIDTH = train_array.shape[1]

In [0]:
hsi_train_data = []
hsi_test_data = []
train_labels = []
test_labels = []
for i in range(HEIGHT):
    for j in range(WIDTH):
        if train_array[i, j] != 0:
            hsi_train_data.append(hsi[:, i, j])
            train_labels.append(train_array[i, j])
        if test_array[i,j] != 0:
            hsi_test_data.append(hsi[:, i, j])
            test_labels.append(test_array[i, j])

In [0]:
hsi_train_data = np.array(hsi_train_data)
train_labels = np.array(train_labels)
test_labels = np.array(test_labels)

In [11]:
print(hsi_train_data.shape)
print(train_labels.shape)
print(test_labels.shape)

(2832, 3)
(2832,)
(12197,)


In [12]:
import keras

train_one_hot = keras.utils.to_categorical(train_labels-1)
test_one_hot = keras.utils.to_categorical(test_labels-1)

Using TensorFlow backend.


In [13]:
print(train_one_hot.shape)
print(test_one_hot.shape)

(2832, 15)
(12197, 15)


In [0]:
HSI_PATCH_SIZE = 27
CONV1 = 500
CONV2 = 100
FC1 = 200
FC2 = 84
LEARNING_RATE = 0.005

In [15]:
padded_hsi = np.lib.pad(hsi, ((0,0), (HSI_PATCH_SIZE//2, HSI_PATCH_SIZE//2), (HSI_PATCH_SIZE//2,HSI_PATCH_SIZE//2)), 'reflect')
print(padded_hsi.shape)

(3, 375, 1931)


In [0]:
def get_patches(data, patch_size, row, column):
  offset = patch_size // 2
  row_low = row - offset
  row_high = row + offset
  col_low = column - offset
  col_high = column + offset
  return data[0:, row_low:row_high + 1, col_low:col_high + 1].reshape(patch_size, patch_size, data.shape[0])

In [0]:
hsi_train_patches = []
hsi_test_patches = []
for i in range(HEIGHT):
  for j in range(WIDTH):
    if train_array[i, j] != 0:
      hsi_train_patches.append(get_patches(padded_hsi, HSI_PATCH_SIZE, i+HSI_PATCH_SIZE//2, j+HSI_PATCH_SIZE//2))
    if test_array[i, j] != 0:
      hsi_test_patches.append(get_patches(padded_hsi, HSI_PATCH_SIZE, i+HSI_PATCH_SIZE//2, j+HSI_PATCH_SIZE//2))

In [18]:
hsi_train_patches = np.array(hsi_train_patches)
hsi_test_patches = np.array(hsi_test_patches)
print(hsi_train_patches.shape)
print(hsi_test_patches.shape)

(2832, 27, 27, 3)
(12197, 27, 27, 3)


In [19]:
from tensorflow.python.keras.models import Sequential
from tensorflow.python.keras.layers import Conv2D, Dense, Flatten
from tensorflow.python.keras.layers import InputLayer
from tensorflow.python.keras.layers import  MaxPooling2D
from tensorflow.python.keras.layers import BatchNormalization, Dropout
from tensorflow.python.keras.optimizers import Adam,SGD

BANDS = hsi_train_patches.shape[3]
NUM_CLS = train_one_hot.shape[1]
BATCH_SIZE = 25

hsi_model = Sequential()
hsi_model.add(InputLayer(input_shape=(HSI_PATCH_SIZE, HSI_PATCH_SIZE, BANDS)))
hsi_model.add(Conv2D(kernel_size=6, strides=3, filters=96, padding='same', activation='relu', name='conv1'))
hsi_model.add(BatchNormalization())
hsi_model.add(MaxPooling2D(pool_size=2, strides=2))
hsi_model.add(Conv2D(kernel_size=5, strides=3, filters=128, padding='same', activation='relu', name='conv2'))
hsi_model.add(BatchNormalization())
hsi_model.add(MaxPooling2D(pool_size=2, strides=2))


hsi_model.add(Flatten())
hsi_model.add(Dense(FC1, activation='relu'))
hsi_model.add(Dropout(0.75))
hsi_model.add(Dense(FC2, activation='relu'))
hsi_model.add(Dropout(0.6))
hsi_model.add(Dense(NUM_CLS, activation='softmax'))

sgd = SGD(lr=LEARNING_RATE, decay=1e-6, momentum=0.9, nesterov=True)
hsi_model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

hsi_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1 (Conv2D)               (None, 9, 9, 96)          10464     
_________________________________________________________________
batch_normalization (BatchNo (None, 9, 9, 96)          384       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 4, 4, 96)          0         
_________________________________________________________________
conv2 (Conv2D)               (None, 2, 2, 128)         307328    
_________________________________________________________________
batch_normalization_1 (Batch (None, 2, 2, 128)         512       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 1, 1, 128)         0         
_________________________________________________________________
flatten (Flatten)            (None, 128)               0         
__________

In [0]:
history = hsi_model.fit(hsi_train_patches, train_one_hot, batch_size=BATCH_SIZE, shuffle=True, epochs=50)

Epoch 1/50
2832/2832 [==============================] - 1s 308us/step - loss: 0.0926 - acc: 0.9728
Epoch 2/50
2832/2832 [==============================] - 1s 310us/step - loss: 0.0962 - acc: 0.9742
Epoch 3/50
2832/2832 [==============================] - 1s 312us/step - loss: 0.1230 - acc: 0.9693
Epoch 4/50
2832/2832 [==============================] - 1s 321us/step - loss: 0.0932 - acc: 0.9749
Epoch 5/50
2832/2832 [==============================] - 1s 317us/step - loss: 0.1164 - acc: 0.9686
Epoch 6/50
2832/2832 [==============================] - 1s 313us/step - loss: 0.0864 - acc: 0.9788
Epoch 7/50
2832/2832 [==============================] - 1s 317us/step - loss: 0.0958 - acc: 0.9742
Epoch 8/50
2832/2832 [==============================] - 1s 317us/step - loss: 0.0826 - acc: 0.9763
Epoch 9/50
2832/2832 [==============================] - 1s 321us/step - loss: 0.1073 - acc: 0.9700
Epoch 10/50
2832/2832 [==============================] - 1s 321us/step - loss: 0.0886 - acc: 0.9756
Epoch 11/

In [0]:
from operator import truediv

def AA_andEachClassAccuracy(confusion_matrix):
    counter = confusion_matrix.shape[0]
    list_diag = np.diag(confusion_matrix)
    list_raw_sum = np.sum(confusion_matrix, axis=1)
    each_acc = np.nan_to_num(truediv(list_diag, list_raw_sum))
    average_acc = np.mean(each_acc)
    return each_acc, average_acc

In [35]:
test_cls = test_labels - 1
prediction = hsi_model.predict(hsi_test_patches).argmax(axis=-1)



from sklearn import metrics, preprocessing

overall_acc = metrics.accuracy_score(prediction, test_cls)
kappa = metrics.cohen_kappa_score(prediction, test_cls)
confusion_matrix = metrics.confusion_matrix(prediction, test_cls)
each_acc, average_acc = AA_andEachClassAccuracy(confusion_matrix) 


print("Overall Accuracy of training sapmles : ",overall_acc)
print("Average Accuracy of training samples : ",average_acc)
print("Kappa statistics of training samples : ",kappa)
print("Each class accuracy of training samples : \n", each_acc)
print("Confusion matrix :\n", confusion_matrix)

Overall Accuracy of training sapmles :  0.5439042387472329
Average Accuracy of training samples :  0.5642114937035048
Kappa statistics of training samples :  0.5049075161911842
Each class accuracy of training samples : 
 [0.72953737 0.56846081 0.72608696 0.63561321 0.81314879 0.80327869
 0.76173709 0.50701754 0.54739539 0.57164869 0.42174629 0.31905465
 0.24071082 0.35827664 0.45945946]
Confusion matrix :
 [[820 117   0 134   0   0   0   0   0   0   0   2   0   0  51]
 [ 40 602   0 243   2   0  20   0   4   0   0   0   0  40 108]
 [  0   2 167   0   0   4  22   4   0   9   0   0   1   0  21]
 [  8 141  72 539   0   0  20   0   1   0   1   0   0   0  66]
 [  0   0  19   0 940   0   0  40  11   0   0  95   0  25  26]
 [  0   8   0   0   0  98   2   0   0   0   0   1   9   0   4]
 [  7   0   0  11   0  17 649  18 125   0  15   1   6   0   3]
 [  0   8  13   0  18  10   2 289  65  37   5  55   3   7  58]
 [  0  17   0  10  45   7 217  31 641  67  94  13  25   2   2]
 [  0   0 139   0   2  

In [0]:
hsi_model.save_weights('Models/hsi_model_weights.h5')